# Preview 3D Generator

Генерирует 3D-превью для событий с помощью Blender.

**Входные данные**: dataset с `payload.json` содержащий:
```json
{
  "events": [
    {
      "event_id": 123,
      "title": "Название события",
      "images": ["https://files.catbox.moe/xxx.jpg", ...]
    }
  ]
}
```

**Выходные данные**: `output.json` с результатами:
```json
{
  "results": [
    {"event_id": 123, "preview_url": "https://files.catbox.moe/yyy.webp", "status": "ok"},
    {"event_id": 456, "preview_url": null, "status": "error", "error": "..."}
  ]
}
```

In [ ]:
import os
import json
import asyncio
import requests
import subprocess
import sys
import textwrap
import random
import shutil
from pathlib import Path

# ================= CONFIGURATION =================
WORK_DIR = Path("/kaggle/working")
INPUT_DIR = WORK_DIR / "input_images"
OUTPUT_DIR = WORK_DIR / "output_previews"
PAYLOAD_FILE = Path("/kaggle/input/preview3d-dataset/payload.json")

# Blender
BLENDER_URL = "https://download.blender.org/release/Blender4.0/blender-4.0.2-linux-x64.tar.xz"
BLENDER_DIR = WORK_DIR / "blender_app"
BLENDER_EXE = BLENDER_DIR / "blender"
SCRIPT_PATH = WORK_DIR / "render_scene.py"

# Limits
MAX_IMAGES_PER_EVENT = 57

print(f"Preview 3D Generator initialized")
print(f"GPU available: {os.path.exists('/dev/nvidia0')}")

In [ ]:
# ================= LOAD PAYLOAD =================

def load_payload():
    """Load events from payload.json"""
    if not PAYLOAD_FILE.exists():
        print(f"ERROR: {PAYLOAD_FILE} not found!")
        return []
    
    with open(PAYLOAD_FILE) as f:
        data = json.load(f)
    
    events = data.get("events", [])
    print(f"Loaded {len(events)} events from payload")
    return events

events = load_payload()
for e in events[:3]:
    print(f"  - Event {e['event_id']}: {e['title'][:50]}... ({len(e.get('images', []))} images)")

In [ ]:
# ================= DOWNLOAD IMAGES =================

def download_images(event_id: int, image_urls: list[str], event_dir: Path) -> list[Path]:
    """Download images for an event, return list of local paths."""
    if event_dir.exists():
        shutil.rmtree(event_dir)
    event_dir.mkdir(parents=True, exist_ok=True)
    
    downloaded = []
    urls_to_download = image_urls[:MAX_IMAGES_PER_EVENT]
    
    for i, url in enumerate(urls_to_download):
        ext = url.split('.')[-1].split('?')[0].lower()
        if ext not in ('jpg', 'jpeg', 'png', 'webp'):
            ext = 'jpg'
        fname = f"{i:03d}.{ext}"
        local_path = event_dir / fname
        
        try:
            r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=30)
            if r.status_code == 200:
                with open(local_path, 'wb') as f:
                    f.write(r.content)
                downloaded.append(local_path)
            else:
                print(f"  WARN: {url} returned {r.status_code}")
        except Exception as e:
            print(f"  ERROR downloading {url}: {e}")
    
    return downloaded

print("Download function ready")

In [ ]:
# ================= SETUP BLENDER =================

def setup_blender():
    """Install Blender dependencies and download binary."""
    print(">>> Setting up Blender...")
    
    # Install dependencies
    subprocess.call(
        "apt-get update -yqq && apt-get install -yqq "
        "libx11-6 libxrender1 libxxf86vm1 libxfixes3 libxi6 libgl1 libglib2.0-0",
        shell=True,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    
    if not BLENDER_EXE.exists():
        print(">>> Downloading Blender binary...")
        subprocess.call(f"wget -q {BLENDER_URL} -O blender.tar.xz", shell=True)
        BLENDER_DIR.mkdir(parents=True, exist_ok=True)
        subprocess.call(f"tar -xf blender.tar.xz -C {BLENDER_DIR} --strip-components=1", shell=True)
        subprocess.call("rm blender.tar.xz", shell=True)
    
    print(f">>> Blender ready at {BLENDER_EXE}")

setup_blender()

In [ ]:
# ================= BLENDER SCRIPT =================

def create_blender_script():
    """Generate the Blender Python script for 3D rendering."""
    script = textwrap.dedent(r'''
    import bpy, os, math, sys, random

    POSTER_HEIGHT = 4.0
    CORNER_RADIUS = 0.06     
    GAP_X_FACTOR = 0.7       
    GAP_Y_DEPTH = 1.0        
    ROTATION_Y = 35          
    IDEAL_START_X = -1.44 
    CAMERA_SAFE_LEFT = -4.1

    def clean_and_setup():
        if bpy.context.object and bpy.context.object.mode != 'OBJECT':
            bpy.ops.object.mode_set(mode='OBJECT')
        bpy.ops.object.select_all(action='SELECT')
        bpy.ops.object.delete()
        for c in [bpy.data.meshes, bpy.data.materials, bpy.data.images]:
            for b in c: c.remove(b)

        scene = bpy.context.scene
        scene.render.engine = 'CYCLES'
        scene.render.film_transparent = True
        scene.render.resolution_x = 1920
        scene.render.resolution_y = 1080
        scene.cycles.samples = 128
        
        scene.render.image_settings.file_format = 'WEBP'
        scene.render.image_settings.color_mode = 'RGBA'
        scene.render.image_settings.quality = 90
        
        try:
            prefs = bpy.context.preferences.addons['cycles'].preferences
            prefs.compute_device_type = 'CUDA'
            prefs.get_devices()
            if prefs.devices: scene.cycles.device = 'GPU'
            else: scene.cycles.device = 'CPU'
        except: scene.cycles.device = 'CPU'
        return scene

    def create_material(filepath, ar):
        mat = bpy.data.materials.new(name="PosterMat")
        mat.use_nodes = True
        nodes = mat.node_tree.nodes
        links = mat.node_tree.links
        nodes.clear()
        
        shader = nodes.new('ShaderNodeBsdfPrincipled')
        shader.inputs['Roughness'].default_value = 0.5
        shader.location = (200, 0)
        
        output = nodes.new('ShaderNodeOutputMaterial')
        output.location = (400, 0)
        links.new(shader.outputs['BSDF'], output.inputs['Surface'])
        
        try: img = bpy.data.images.load(filepath)
        except: return None
        tex = nodes.new('ShaderNodeTexImage')
        tex.image = img
        tex.location = (0, 200)
        links.new(tex.outputs['Color'], shader.inputs['Base Color'])
        
        scale_x = ar if ar >= 1.0 else 1.0
        scale_y = 1.0 if ar >= 1.0 else 1.0/ar
        
        uv = nodes.new('ShaderNodeUVMap')
        uv.location = (-1200, 0)
        
        vec_sub = nodes.new('ShaderNodeVectorMath')
        vec_sub.operation = 'SUBTRACT'
        vec_sub.inputs[1].default_value = (0.5, 0.5, 0)
        vec_sub.location = (-1000, 0)
        links.new(uv.outputs['UV'], vec_sub.inputs[0])
        
        vec_abs = nodes.new('ShaderNodeVectorMath')
        vec_abs.operation = 'ABSOLUTE'
        vec_abs.location = (-800, 0)
        links.new(vec_sub.outputs['Vector'], vec_abs.inputs[0])

        vec_scale = nodes.new('ShaderNodeVectorMath')
        vec_scale.operation = 'MULTIPLY'
        vec_scale.inputs[1].default_value = (scale_x, scale_y, 1.0)
        vec_scale.location = (-600, 0)
        links.new(vec_abs.outputs['Vector'], vec_scale.inputs[0])

        limit_x = (scale_x * 0.5) - CORNER_RADIUS
        limit_y = (scale_y * 0.5) - CORNER_RADIUS
        
        vec_sub_rad = nodes.new('ShaderNodeVectorMath')
        vec_sub_rad.operation = 'SUBTRACT'
        vec_sub_rad.inputs[1].default_value = (limit_x, limit_y, 0)
        vec_sub_rad.location = (-400, 0)
        links.new(vec_scale.outputs['Vector'], vec_sub_rad.inputs[0])
        
        vec_max = nodes.new('ShaderNodeVectorMath')
        vec_max.operation = 'MAXIMUM'
        vec_max.inputs[1].default_value = (0, 0, 0)
        vec_max.location = (-200, 0)
        links.new(vec_sub_rad.outputs['Vector'], vec_max.inputs[0])
        
        vec_len = nodes.new('ShaderNodeVectorMath')
        vec_len.operation = 'LENGTH'
        vec_len.location = (0, -200)
        links.new(vec_max.outputs['Vector'], vec_len.inputs[0])
        
        math_gt = nodes.new('ShaderNodeMath')
        math_gt.operation = 'GREATER_THAN'
        math_gt.inputs[1].default_value = CORNER_RADIUS
        math_gt.location = (200, -200)
        links.new(vec_len.outputs['Value'], math_gt.inputs[0])
        
        math_inv = nodes.new('ShaderNodeMath')
        math_inv.operation = 'SUBTRACT'
        math_inv.inputs[0].default_value = 1.0
        math_inv.location = (400, -200)
        links.new(math_gt.outputs['Value'], math_inv.inputs[1])

        links.new(math_inv.outputs['Value'], shader.inputs['Alpha'])
        return mat

    def build(image_dir, output_path):
        scene = clean_and_setup()
        master_col = scene.collection
        
        cam = bpy.data.objects.new('Camera', bpy.data.cameras.new('Camera'))
        master_col.objects.link(cam)
        cam.location = (0, -12, 0)
        cam.rotation_euler = (math.radians(90), 0, 0)
        cam.data.lens = 50
        scene.camera = cam
        
        sun = bpy.data.objects.new('Sun', bpy.data.lights.new('Sun', 'SUN'))
        sun.data.energy = 3.0
        master_col.objects.link(sun)
        sun.rotation_euler = (math.radians(45), math.radians(15), math.radians(45))
        
        fill = bpy.data.objects.new('Fill', bpy.data.lights.new('Fill', 'AREA'))
        fill.data.energy = 400
        master_col.objects.link(fill)
        fill.location = (-5, -5, 2)
        fill.rotation_euler = (math.radians(60), 0, math.radians(-45))

        if not os.path.exists(image_dir): return
        
        files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg', '.webp'))])
        if not files: return
        
        random.shuffle(files)
        print(f"DEBUG: Randomized Order -> {files}")

        current_x = IDEAL_START_X
        current_y = 0.0
        prev_half_w = 0
        
        for i, f in enumerate(files):
            path = os.path.join(image_dir, f)
            try:
                tmp = bpy.data.images.load(path)
                w, h = tmp.size
                ar = w / h
                bpy.data.images.remove(tmp)
            except: continue
            
            w_3d = POSTER_HEIGHT * ar
            
            if i == 0:
                half_w_current = w_3d / 2
                left_edge = IDEAL_START_X - half_w_current
                if left_edge < CAMERA_SAFE_LEFT:
                    offset = CAMERA_SAFE_LEFT - left_edge
                    current_x = IDEAL_START_X + offset
                    print(f"DEBUG: Shifting scene right by {offset:.2f} for wide image")
                else:
                    current_x = IDEAL_START_X
            
            bpy.ops.mesh.primitive_plane_add(size=1)
            plane = bpy.context.active_object
            if plane.name not in master_col.objects:
                try: master_col.objects.link(plane)
                except: pass
                
            plane.name = f"Poster_{i}"
            plane.scale = (w_3d, POSTER_HEIGHT, 1)
            bpy.ops.object.transform_apply(scale=True)
            
            mat = create_material(path, ar)
            if mat: plane.data.materials.append(mat)
            
            if i == 0:
                plane.location = (current_x, 0, 0)
                plane.rotation_euler = (math.radians(90), 0, 0)
                prev_half_w = w_3d / 2
            else:
                shift = (prev_half_w + (w_3d / 2)) * GAP_X_FACTOR
                current_x += shift
                current_y += GAP_Y_DEPTH
                plane.location = (current_x, current_y, 0)
                plane.rotation_euler = (math.radians(90), 0, math.radians(-ROTATION_Y))
                prev_half_w = w_3d / 2

        scene.render.filepath = output_path
        bpy.ops.render.render(write_still=True)

    if __name__ == "__main__":
        try:
            idx = sys.argv.index("--")
            build(sys.argv[idx+1], sys.argv[idx+2])
        except: sys.exit(1)
    ''')
    
    with open(SCRIPT_PATH, "w") as f:
        f.write(script)
    print(f">>> Blender script saved to {SCRIPT_PATH}")

create_blender_script()

In [ ]:
# ================= UPLOAD TO CATBOX =================

def upload_to_catbox(filepath: Path) -> str | None:
    """Upload file to catbox.moe and return URL."""
    try:
        with open(filepath, 'rb') as f:
            files = {'fileToUpload': (filepath.name, f)}
            data = {'reqtype': 'fileupload'}
            r = requests.post(
                'https://catbox.moe/user/api.php',
                data=data,
                files=files,
                timeout=120
            )
            if r.status_code == 200 and r.text.startswith('https://'):
                print(f"  CATBOX OK: {r.text}")
                return r.text.strip()
            else:
                print(f"  CATBOX FAIL: {r.status_code} - {r.text[:100]}")
                return None
    except Exception as e:
        print(f"  CATBOX ERROR: {e}")
        return None

print("Catbox upload function ready")

In [ ]:
# ================= RENDER EVENT =================

def render_event(event: dict) -> dict:
    """Render 3D preview for a single event."""
    event_id = event['event_id']
    title = event.get('title', 'Unknown')[:50]
    images = event.get('images', [])
    
    print(f"\n{'='*60}")
    print(f"Processing event {event_id}: {title}")
    print(f"Images: {len(images)}")
    
    if not images:
        return {'event_id': event_id, 'preview_url': None, 'status': 'skip', 'error': 'No images'}
    
    # Download images
    event_dir = INPUT_DIR / str(event_id)
    downloaded = download_images(event_id, images, event_dir)
    print(f"Downloaded: {len(downloaded)} images")
    
    if not downloaded:
        return {'event_id': event_id, 'preview_url': None, 'status': 'error', 'error': 'No images downloaded'}
    
    # Render
    output_file = OUTPUT_DIR / f"{event_id}.webp"
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    cmd = [
        str(BLENDER_EXE), "-b", "--factory-startup",
        "-P", str(SCRIPT_PATH),
        "--", str(event_dir), str(output_file)
    ]
    
    print(f"Running Blender...")
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        if result.returncode != 0:
            print(f"Blender stderr: {result.stderr[-500:]}")
            return {'event_id': event_id, 'preview_url': None, 'status': 'error', 'error': 'Blender failed'}
    except subprocess.TimeoutExpired:
        return {'event_id': event_id, 'preview_url': None, 'status': 'error', 'error': 'Render timeout'}
    except Exception as e:
        return {'event_id': event_id, 'preview_url': None, 'status': 'error', 'error': str(e)}
    
    if not output_file.exists():
        return {'event_id': event_id, 'preview_url': None, 'status': 'error', 'error': 'Output file not created'}
    
    print(f"Render complete: {output_file.stat().st_size / 1024:.1f} KB")
    
    # Upload to catbox
    preview_url = upload_to_catbox(output_file)
    if not preview_url:
        return {'event_id': event_id, 'preview_url': None, 'status': 'error', 'error': 'Catbox upload failed'}
    
    return {'event_id': event_id, 'preview_url': preview_url, 'status': 'ok'}

print("Render function ready")

In [ ]:
# ================= MAIN PROCESSING =================

results = []

for i, event in enumerate(events):
    print(f"\n>>> Processing {i+1}/{len(events)}")
    result = render_event(event)
    results.append(result)
    print(f"Result: {result['status']}")

# Summary
ok_count = sum(1 for r in results if r['status'] == 'ok')
error_count = sum(1 for r in results if r['status'] == 'error')
skip_count = sum(1 for r in results if r['status'] == 'skip')

print(f"\n{'='*60}")
print(f"SUMMARY: {len(results)} events processed")
print(f"  OK: {ok_count}")
print(f"  Errors: {error_count}")
print(f"  Skipped: {skip_count}")

In [ ]:
# ================= SAVE OUTPUT =================

output_data = {
    'results': results,
    'summary': {
        'total': len(results),
        'ok': ok_count,
        'errors': error_count,
        'skipped': skip_count
    }
}

output_json_path = WORK_DIR / 'output.json'
with open(output_json_path, 'w') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"Output saved to {output_json_path}")
print("\nDone!")